#### Table of contents
* [Day 1: Inverse Captcha](#----Day-1:-Inverse-Captcha----)
* [Day 2: Corruption Checksum](#----Day-2:-Corruption-Checksum----)

Let's prepare some imports and helper function, that will allow us later to work with common problems.

In [85]:
# Python 3.x
from itertools import combinations, permutations

def _input(day):
    "Open this day's input file."
    filename = 'data/input{}.txt'.format(day)
    return open(filename)

def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))

### --- Day 1: Inverse Captcha ---
The captcha requires you to review a sequence of digits (your puzzle input) and find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.


For example:

    1122 produces a sum of 3 (1 + 2) because the first digit (1) matches the second digit and the third digit (2) matches the fourth digit.
    1111 produces 4 because each digit (all 1) matches the next.
    1234 produces 0 because no digit matches the next.
    91212129 produces 9 because the only digit that matches the next one is the last digit, 9.


Initially, I attempted to look at the list as something we need to rotate and iterate through. It became increasingly difficult for corner cases e.g. to address the 91212129 (9) case because the only digit that matches the next one is the last digit, 9. I spent way too much time overthinking it, when I should have re-focused and thought about the remainder and modulo in the beginning.

In [2]:
array = '91212129'
chars = len(array)
print(array, chars)

res = []
for c, n in enumerate(array):
    # The % (modulo) operator yields the remainder from the division of the first argument by the second. 
    # So in case where `array[(c + 1) % chars]` is the last char, `array[(7+1)%8]` will yield 0 because 
    # there is no remainder, which means we've reached the end of the list. This will get us the 
    # first element in the array, `array[0]` as if starting iteration from the first one.
    print("counter: {}, number: {}, next: {}, same as prev: {}".format(
        c, n, array[(c + 1) % chars], n == array[(c + 1) % chars]))
    if n == array[(c + 1) % chars]:
        res.append(int(n))

print(sum(res))

91212129 8
counter: 0, number: 9, next: 1, same as prev: False
counter: 1, number: 1, next: 2, same as prev: False
counter: 2, number: 2, next: 1, same as prev: False
counter: 3, number: 1, next: 2, same as prev: False
counter: 4, number: 2, next: 1, same as prev: False
counter: 5, number: 1, next: 2, same as prev: False
counter: 6, number: 2, next: 9, same as prev: False
counter: 7, number: 9, next: 9, same as prev: True
9


Now let's simplify and run the above solution through the example cases and a few custom ones.

In [3]:
def inverse_captcha(array): 
    return sum(int(n) for c, n in enumerate(array) if n == array[(c + 1) % len(array)])

assert inverse_captcha('1122') == 3
assert inverse_captcha('1111') == 4
assert inverse_captcha('1234') == 0
assert inverse_captcha('91212129') == 9
assert inverse_captcha('08798780') == 0
assert inverse_captcha('3335553') == 19
assert inverse_captcha('3' * 10) == 30
assert inverse_captcha('1' * 10 + '7' * 130 + '5' * 25 ) == 1032

Now let's run it for for the input:

In [83]:
input_array = _input(1).read().strip()
print(inverse_captcha(input_array))

1175


#### --- Part Two ---
Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list. That is, if your list contains 10 items, only include a digit in your sum if the digit 10/2 = 5 steps forward matches it. Fortunately, your list has an even number of elements.

For example:

    1212 produces 6: the list contains 4 items, and all four digits match the digit 2 items ahead.
    1221 produces 0, because every comparison is between a 1 and a 2.
    123425 produces 4, because both 2s match each other, but no other digit has a match.
    123123 produces 12.
    12131415 produces 4.

Here, we only need to modify the `array[(c + 1) % len(array)]` part to include slicing list by half, such that it yields the remainder in the same way it did above. So the array part becomes ``array[(c + len(array) // 2) % len(array)]``

In [5]:
array = '123123'

sum(int(n) for c, n in enumerate(array) if n == array[(c + len(array) // 2) % len(array)])

12

Let's run it through the tests again. Before we'll try the function on the problem's input.

In [6]:
def inverse_captcha_2(array): 
    return sum(int(n) for c, n in enumerate(array) if n == array[(c + len(array) // 2) % len(array)])

assert inverse_captcha_2('1212') == 6
assert inverse_captcha_2('1221') == 0
assert inverse_captcha_2('123425') == 4
assert inverse_captcha_2('123123') == 12
assert inverse_captcha_2('123425') == 4
assert inverse_captcha_2('66' * 3 + '123' + '66' * 3) == 54

In [7]:
inverse_captcha_2(input_array)

1166

### --- Day 2: Corruption Checksum ---

The spreadsheet consists of rows of apparently-random numbers. To make sure the recovery process is on the right track, they need you to calculate the spreadsheet's checksum. For each row, determine the difference between the largest value and the smallest value; the checksum is the sum of all of these differences.

For example, given the following spreadsheet:

```
5 1 9 5
7 5 3 
2 4 6 8
```

    The first row's largest and smallest values are 9 and 1, and their difference is 8.
    The second row's largest and smallest values are 7 and 3, and their difference is 4.
    The third row's difference is 6.

In this example, the spreadsheet's checksum would be 8 + 4 + 6 = 18.


In [8]:
lines = [l.split() for l in _input('2sample').readlines()]
print(lines)

[['5', '1', '9', '5'], ['7', '5', '3'], ['2', '4', '6', '8']]


In [9]:
for line in lines:
    print("max: {}, min: {}, dif: {}, line {}".format(
        max(line), min(line), int(max(line))-int(min(line)), line))

max: 9, min: 1, dif: 8, line ['5', '1', '9', '5']
max: 7, min: 3, dif: 4, line ['7', '5', '3']
max: 8, min: 2, dif: 6, line ['2', '4', '6', '8']


In [10]:
sum(int(max(line)) - int(min(line)) for line in lines)

18

In [11]:
input_lines = [i.split() for i in _input(2).readlines()]
print(input_lines)

[['1364', '461', '1438', '1456', '818', '999', '105', '1065', '314', '99', '1353', '148', '837', '590', '404', '123'], ['204', '99', '235', '2281', '2848', '3307', '1447', '3848', '3681', '963', '3525', '525', '288', '278', '3059', '821'], ['280', '311', '100', '287', '265', '383', '204', '380', '90', '377', '398', '99', '194', '297', '399', '87'], ['7698', '2334', '7693', '218', '7344', '3887', '3423', '7287', '7700', '2447', '7412', '6147', '231', '1066', '248', '208'], ['3740', '837', '4144', '123', '155', '2494', '1706', '4150', '183', '4198', '1221', '4061', '95', '148', '3460', '550'], ['1376', '1462', '73', '968', '95', '1721', '544', '982', '829', '1868', '1683', '618', '82', '1660', '83', '1778'], ['197', '2295', '5475', '2886', '2646', '186', '5925', '237', '3034', '5897', '1477', '196', '1778', '3496', '5041', '3314'], ['179', '2949', '3197', '2745', '1341', '3128', '1580', '184', '1026', '147', '2692', '212', '2487', '2947', '3547', '1120'], ['460', '73', '52', '373', '41',

In [12]:
sum(int(max(line)) - int(min(line)) for line in input_lines)

27517

Looks like the answer is not quite right. Let's run some tests and try to investigate.

In [13]:
for line in input_lines:
    print("max: {}, min: {}, dif: {}, line {}".format(
        max(line), min(line), int(max(line))-int(min(line)), line))

max: 999, min: 105, dif: 894, line ['1364', '461', '1438', '1456', '818', '999', '105', '1065', '314', '99', '1353', '148', '837', '590', '404', '123']
max: 99, min: 1447, dif: -1348, line ['204', '99', '235', '2281', '2848', '3307', '1447', '3848', '3681', '963', '3525', '525', '288', '278', '3059', '821']
max: 99, min: 100, dif: -1, line ['280', '311', '100', '287', '265', '383', '204', '380', '90', '377', '398', '99', '194', '297', '399', '87']
max: 7700, min: 1066, dif: 6634, line ['7698', '2334', '7693', '218', '7344', '3887', '3423', '7287', '7700', '2447', '7412', '6147', '231', '1066', '248', '208']
max: 95, min: 1221, dif: -1126, line ['3740', '837', '4144', '123', '155', '2494', '1706', '4150', '183', '4198', '1221', '4061', '95', '148', '3460', '550']
max: 982, min: 1376, dif: -394, line ['1376', '1462', '73', '968', '95', '1721', '544', '982', '829', '1868', '1683', '618', '82', '1660', '83', '1778']
max: 5925, min: 1477, dif: 4448, line ['197', '2295', '5475', '2886', '264

Looks like from the first line the `max()` is wrong. It's returning `max: 999` where in line there are numbers over thousand: `['1364', '461', '1438', '1456', '818', '999', '105', '1065', '314', '99', '1353', '148', '837', '590', '404', '123']`

In [14]:
max(['1364', '461', '1438', '1456', '818', '999', '105', '1065', 
     '314', '99', '1353', '148', '837', '590', '404', '123']) 

'999'

In [15]:
# The optional key argument specifies a one-argument ordering function like that used for list.sort()
# To address the above case, we need to think of list items as numbers, 
# by providing key to max function as int will return the correct maximum result.
max(['1364', '461', '1438', '1456', '818', '999', '105', '1065', 
     '314', '99', '1353', '148', '837', '590', '404', '123'], key=int) 

'1456'

Now, let's run some tests:

In [16]:
def corruption_checksum(lines):
    return sum(int(max(line, key=int)) - int(min(line, key=int)) for line in lines)

assert corruption_checksum([['5', '1', '9', '5']]) == 8
assert corruption_checksum([['1364', '999', '123']]) == 1241
assert corruption_checksum([['5', '1', '9', '5'], ['7', '5', '3'], ['2', '4', '6', '8']]) == 18
assert corruption_checksum([['204', '99', '235', '2281', '2848', '3307']]) == 3208

So we can either cast all elements of the list into numbers, or use the key in max set to int.

In [17]:
corruption_checksum(input_lines)

53460

#### --- Part Two ---
It sounds like the goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. They would like you to find those numbers on each line, divide them, and add up each line's result.

For example, given the following spreadsheet:
```
5 9 2 8
9 4 7 3
3 8 6 5
```
    In the first row, the only two numbers that evenly divide are 8 and 2; the result of this division is 4.
    In the second row, the two numbers are 9 and 3; the result is 3.
    In the third row, the result is 2.

In this example, the sum of the results would be 4 + 3 + 2 = 9.

In [68]:
test_input = [['5', '9', '2', '8'], ['9', '4', '7', '3'], ['3', '8', '6', '5']]

# let's try the brute force solution first
res = []
arr = list(map(int, ['5', '9', '2', '8']))

# by creating permutations from the array
perm = [i for i in permutations(arr, 2)]
print(perm)

# and then checking if the modulo remainder is 0, will give us the pair
print([i for i in perm if i[0]%i[1]==0])

# from which we can get the two numbers that are evenly divisible between each other
print([i[0]//i[1] for i in perm if i[0]%i[1]==0])

res = []
for l in test_input:
    p = [i for i in permutations(list(map(int, l)), 2)]
    print(p)
    print([i[0]//i[1] for i in p if i[0]%i[1]==0])
    res.append([i[0]//i[1] for i in p if i[0]%i[1]==0][0])
    
print(sum(res))

[(5, 9), (5, 2), (5, 8), (9, 5), (9, 2), (9, 8), (2, 5), (2, 9), (2, 8), (8, 5), (8, 9), (8, 2)]
[(8, 2)]
[4]
[(5, 9), (5, 2), (5, 8), (9, 5), (9, 2), (9, 8), (2, 5), (2, 9), (2, 8), (8, 5), (8, 9), (8, 2)]
[4]
[(9, 4), (9, 7), (9, 3), (4, 9), (4, 7), (4, 3), (7, 9), (7, 4), (7, 3), (3, 9), (3, 4), (3, 7)]
[3]
[(3, 8), (3, 6), (3, 5), (8, 3), (8, 6), (8, 5), (6, 3), (6, 8), (6, 5), (5, 3), (5, 8), (5, 6)]
[2]
9


Let's run some tests to make sure we're not cutting corners here. Perhaps some optimisation is also due for the function.

In [82]:
def corruption_checksum_2(lines):
    return sum([i[0]//i[1] for i in permutations(list(map(int, l)), 2) if i[0]%i[1]==0][0] for l in lines)

assert corruption_checksum_2(test_input) == 9
assert corruption_checksum_2([['5', '9', '2', '8', '8']]) == 4
assert corruption_checksum_2([['5', '9', '2', '8', '4']]) == 4
assert corruption_checksum_2([['3', '1']]) == 3
assert corruption_checksum_2([['30', '10']]) == 3
assert corruption_checksum_2([['500', '10', '5']]) == 50

50

In [76]:
corruption_checksum_2(input_lines)

282